In [2]:
from pymongo import MongoClient
import pandas as pd

Import dataset

In [6]:
Netflix = pd.read_csv(r"Desktop/Netflix_cleaned.csv")

Connect to MongoDB 

In [12]:
C = MongoClient("mongodb://localhost:27017/")
db = C["Pythondb"]
col = db["Netflix"]

In [13]:
col.insert_many(Netflix.to_dict("records"))

InsertManyResult([ObjectId('68a0d95bc2af083e5bcf4f78'), ObjectId('68a0d95bc2af083e5bcf4f79'), ObjectId('68a0d95bc2af083e5bcf4f7a'), ObjectId('68a0d95bc2af083e5bcf4f7b'), ObjectId('68a0d95bc2af083e5bcf4f7c'), ObjectId('68a0d95bc2af083e5bcf4f7d'), ObjectId('68a0d95bc2af083e5bcf4f7e'), ObjectId('68a0d95bc2af083e5bcf4f7f'), ObjectId('68a0d95bc2af083e5bcf4f80'), ObjectId('68a0d95bc2af083e5bcf4f81'), ObjectId('68a0d95bc2af083e5bcf4f82'), ObjectId('68a0d95bc2af083e5bcf4f83'), ObjectId('68a0d95bc2af083e5bcf4f84'), ObjectId('68a0d95bc2af083e5bcf4f85'), ObjectId('68a0d95bc2af083e5bcf4f86'), ObjectId('68a0d95bc2af083e5bcf4f87'), ObjectId('68a0d95bc2af083e5bcf4f88'), ObjectId('68a0d95bc2af083e5bcf4f89'), ObjectId('68a0d95bc2af083e5bcf4f8a'), ObjectId('68a0d95bc2af083e5bcf4f8b'), ObjectId('68a0d95bc2af083e5bcf4f8c'), ObjectId('68a0d95bc2af083e5bcf4f8d'), ObjectId('68a0d95bc2af083e5bcf4f8e'), ObjectId('68a0d95bc2af083e5bcf4f8f'), ObjectId('68a0d95bc2af083e5bcf4f90'), ObjectId('68a0d95bc2af083e5bcf4f

In [16]:
pipeline = [
    {"$unwind": "$listed_in"},
    {"$group": {"_id": "$listed_in", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 100}}},
    {"$sort": {"count": -1}}
]

In [21]:
result = list(col.aggregate(pipeline))
print(result)

[{'_id': 'Documentaries', 'count': 712}, {'_id': 'Dramas, International Movies', 'count': 709}, {'_id': 'Stand-Up Comedy', 'count': 654}, {'_id': 'Comedies, Dramas, International Movies', 'count': 533}, {'_id': 'Dramas, Independent Movies, International Movies', 'count': 496}, {'_id': "Kids' TV", 'count': 431}, {'_id': 'Children & Family Movies', 'count': 421}, {'_id': 'Children & Family Movies, Comedies', 'count': 392}, {'_id': 'Documentaries, International Movies', 'count': 369}, {'_id': 'Dramas, International Movies, Romantic Movies', 'count': 348}, {'_id': 'Comedies, International Movies', 'count': 345}, {'_id': 'Comedies, International Movies, Romantic Movies', 'count': 297}, {'_id': 'Dramas, International Movies, Thrillers', 'count': 267}, {'_id': 'Dramas', 'count': 258}, {'_id': 'Action & Adventure, Dramas, International Movies', 'count': 248}, {'_id': 'International TV Shows, TV Dramas', 'count': 242}, {'_id': 'Action & Adventure', 'count': 239}, {'_id': 'Comedies, Dramas, Inde

In [22]:
pipeline = [
    {"$match": {"type": "Movie", "release_year": {"$gt": 2015}}},
    {"$unwind": "$country"},
    {"$group": {"_id": "$country", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]

In [24]:
result = list(col.aggregate(pipeline))
print(result)

[{'_id': 'United States', 'count': 2366}, {'_id': 'India', 'count': 812}, {'_id': 'unknown', 'count': 686}, {'_id': 'United Kingdom', 'count': 280}, {'_id': 'Canada', 'count': 186}]


In [37]:
pipeline = [
    {
        "$match": {
            "date_added": {
                "$type": "string"  
            }
        }
    },
    {
        "$addFields": {
            "date_added": {
                "$dateFromString": {
                    "dateString": "$date_added",
                    "format": "%Y-%m-%d"  
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "year": {"$year": "$date_added"},
                "month": {"$month": "$date_added"}
            },
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"_id": 1}
    }
]

result = list(col.aggregate(pipeline))
print(result)

[{'_id': {'year': 2008, 'month': 1}, 'count': 2}, {'_id': {'year': 2008, 'month': 2}, 'count': 2}, {'_id': {'year': 2009, 'month': 5}, 'count': 2}, {'_id': {'year': 2009, 'month': 11}, 'count': 2}, {'_id': {'year': 2010, 'month': 11}, 'count': 1}, {'_id': {'year': 2011, 'month': 5}, 'count': 2}, {'_id': {'year': 2011, 'month': 9}, 'count': 2}, {'_id': {'year': 2011, 'month': 10}, 'count': 11}, {'_id': {'year': 2012, 'month': 2}, 'count': 2}, {'_id': {'year': 2012, 'month': 11}, 'count': 2}, {'_id': {'year': 2012, 'month': 12}, 'count': 2}, {'_id': {'year': 2013, 'month': 3}, 'count': 2}, {'_id': {'year': 2013, 'month': 8}, 'count': 2}, {'_id': {'year': 2013, 'month': 9}, 'count': 4}, {'_id': {'year': 2013, 'month': 10}, 'count': 6}, {'_id': {'year': 2013, 'month': 11}, 'count': 4}, {'_id': {'year': 2013, 'month': 12}, 'count': 4}, {'_id': {'year': 2014, 'month': 1}, 'count': 4}, {'_id': {'year': 2014, 'month': 2}, 'count': 4}, {'_id': {'year': 2014, 'month': 4}, 'count': 4}, {'_id': {'

In [38]:
pipeline = [
    {"$group": {"_id": {"type": "$type", "rating": "$rating"}, "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$group": {
        "_id": "$_id.type",
        "rating": {"$first": "$_id.rating"},
        "count": {"$first": "$count"}
    }}
]
result = list(col.aggregate(pipeline))
print(result)

[{'_id': 'Movie', 'rating': 'TV-14', 'count': 2734}, {'_id': 'TV Show', 'rating': 'TV-14', 'count': 1453}]


In [39]:
db.titles.aggregate([
    {"$group": {"_id": "$title", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1}}},
    {"$sort": {"count": -1}}
])
result = list(col.aggregate(pipeline))
print(result)

[{'_id': 'Movie', 'rating': 'TV-14', 'count': 2734}, {'_id': 'TV Show', 'rating': 'TV-14', 'count': 1453}]


In [40]:
Netflix.to_csv('Netflix.csv', index=False)